In [1]:
import os
import re
import MeCab
import pickle
import gensim
import ast
import lda
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import gensim.corpora as corpora
from gensim import similarities 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import spacy
from pprint import pprint
import warnings
import logging
from google.cloud import storage as gcs
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist 
from scipy.spatial.distance import squareform
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
from collections import Counter
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
%matplotlib inline

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message


In [2]:
INPUT_BUCKET = 'gs://mj-labeling-questionnaires-dev.datasets.linkbal.com/inputs/ver_01/'
PROJECT_NAME = 'linkbal-dp'
BUCKET_NAME = 'mj-labeling-questionnaires-dev.datasets.linkbal.com' 
client = gcs.Client(PROJECT_NAME)
bucket = gcs.Bucket(client, name=BUCKET_NAME)
jp_stop_words = open("/Users/jesse.ulundo/demo_tests/Japanese_stopword_list.txt", "r")
qs = pd.read_csv(INPUT_BUCKET + 'mj_questionnaire_text.csv', sep=',')

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/



In [3]:
qs = qs.drop(['overall_satisfaction', 'overall_satisfaction_reason'], axis=1)
qs = qs.dropna()
qs = qs.drop_duplicates() 
qs

,id,want_recommend,want_recommend_reason
4,111376,10.0,主催のハッピーさんが、とても良かったです。\nまた参加した時にハッピーさんが居ると安心できる...
5,111377,2.0,男女比がおかしい（女性が少ない）
7,111379,3.0,正直、周りの常連さん雰囲気が一人参加だと溶け込みにくいあるグループが生まれた。
8,111380,7.0,楽しめたから
11,111383,0.0,主催者の司会が雑だし、かつ暴言を聞こえない程度にボソッと言われた。
...,...,...,...
20304,131672,8.0,とても楽しい交流会だったので！\nただ、テーブルの回転は少ないので交流しきれない面も出てきます。
20305,131673,1.0,相手の人が少なくて暇な時間がさみしかったです。
20307,131675,0.0,予約確認メールも届いているのに席が用意されていなかった。\nあまりの対応に参加辞退を申し出た...
20308,131676,3.0,段取りがあまりよくなくて、人数の割合も合わず、少人数過ぎた。


In [4]:
qs = qs[qs.want_recommend < 5]
qs = qs[qs.want_recommend_reason != '上記同様']
qs = qs[qs.want_recommend_reason != '同上']
qs

,id,want_recommend,want_recommend_reason
5,111377,2.0,男女比がおかしい（女性が少ない）
7,111379,3.0,正直、周りの常連さん雰囲気が一人参加だと溶け込みにくいあるグループが生まれた。
11,111383,0.0,主催者の司会が雑だし、かつ暴言を聞こえない程度にボソッと言われた。
12,111384,0.0,色々おかしい
16,111388,3.0,時間半分でいいので全員と話したい
...,...,...,...
20301,131669,3.0,自己紹介カード微妙
20305,131673,1.0,相手の人が少なくて暇な時間がさみしかったです。
20307,131675,0.0,予約確認メールも届いているのに席が用意されていなかった。\nあまりの対応に参加辞退を申し出た...
20308,131676,3.0,段取りがあまりよくなくて、人数の割合も合わず、少人数過ぎた。


In [5]:
tagger = MeCab.Tagger()
def tokenize_stemmer(text):
    lemmas = []
    for line in tagger.parse(text).splitlines()[:-1]:
        surface, feature = line.split('\t')
        if feature.split(',')[6] != '*':
            lemmas.append(feature.split(',')[6])
    return lemmas

In [6]:
categories = ['参加人数に関して', '男女比率に関して', 'スタッフ対応に関して', '飲食内容に関して', '虚偽記載']
categories_tkn = [tokenize_stemmer(text) for text in categories]
categories_tkn

[['参加', '人数', 'に関して'],
 ['男女', '比率', 'に関して'],
 ['スタッフ', '対応', 'に関して'],
 ['飲食', '内容', 'に関して'],
 ['虚偽', '記載']]

In [7]:
qs['tokenized_want_recommend_reason_reason'] = qs.apply(lambda row: tokenize_stemmer(row['want_recommend_reason']), axis=1)
qs

,id,want_recommend,want_recommend_reason,tokenized_want_recommend_reason_reason
5,111377,2.0,男女比がおかしい（女性が少ない）,"[男女, 比, が, おかしい, （, 女性, が, 少ない, ）]"
7,111379,3.0,正直、周りの常連さん雰囲気が一人参加だと溶け込みにくいあるグループが生まれた。,"[正直, 、, 周り, の, 常連, さん, 雰囲気, が, 一, 人, 参加, だ, と,..."
11,111383,0.0,主催者の司会が雑だし、かつ暴言を聞こえない程度にボソッと言われた。,"[主催, 者, の, 司会, が, 雑, だ, し, 、, かつ, 暴言, を, 聞こえる,..."
12,111384,0.0,色々おかしい,"[色々, おかしい]"
16,111388,3.0,時間半分でいいので全員と話したい,"[時間, 半分, だ, いい, ので, 全員, と, 話す, たい]"
...,...,...,...,...
20301,131669,3.0,自己紹介カード微妙,"[自己, 紹介, カード, 微妙]"
20305,131673,1.0,相手の人が少なくて暇な時間がさみしかったです。,"[相手, の, 人, が, 少ない, て, 暇, だ, 時間, が, さみしい, た, です..."
20307,131675,0.0,予約確認メールも届いているのに席が用意されていなかった。\nあまりの対応に参加辞退を申し出た...,"[予約, 確認, メール, も, 届く, て, いる, のに, 席, が, 用意, する, ..."
20308,131676,3.0,段取りがあまりよくなくて、人数の割合も合わず、少人数過ぎた。,"[段取り, が, あまり, よい, ない, て, 、, 人数, の, 割合, も, 合う, ..."


In [8]:
stop = [words.strip("\n") for words in jp_stop_words]
qs['tokenized_want_recommend_reason_reason'] = qs['tokenized_want_recommend_reason_reason'].apply(lambda x: [item for item in x if item not in stop])
qs

,id,want_recommend,want_recommend_reason,tokenized_want_recommend_reason_reason
5,111377,2.0,男女比がおかしい（女性が少ない）,"[男女, 比, おかしい, 女性, 少ない]"
7,111379,3.0,正直、周りの常連さん雰囲気が一人参加だと溶け込みにくいあるグループが生まれた。,"[正直, 周り, 常連, 雰囲気, 参加, 溶け込む, にくい, グループ, 生まれる]"
11,111383,0.0,主催者の司会が雑だし、かつ暴言を聞こえない程度にボソッと言われた。,"[主催, 司会, 雑, かつ, 暴言, 聞こえる, 程度, ボソッ, 言う]"
12,111384,0.0,色々おかしい,"[色々, おかしい]"
16,111388,3.0,時間半分でいいので全員と話したい,"[半分, 全員, 話す]"
...,...,...,...,...
20301,131669,3.0,自己紹介カード微妙,"[自己, 紹介, カード, 微妙]"
20305,131673,1.0,相手の人が少なくて暇な時間がさみしかったです。,"[相手, 少ない, 暇, さみしい]"
20307,131675,0.0,予約確認メールも届いているのに席が用意されていなかった。\nあまりの対応に参加辞退を申し出た...,"[予約, 確認, メール, 届く, 用意, あまり, 対応, 参加, 辞退, 申し出る, 悪..."
20308,131676,3.0,段取りがあまりよくなくて、人数の割合も合わず、少人数過ぎた。,"[段取り, あまり, 人数, 割合, 合う, 少, 人数, 過ぎる]"


In [9]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [10]:
def get_corpus(df):
    #df['text'] = strip_newline(df.text)
    words = stop
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [13]:
qs_new = qs[pd.notnull(qs['tokenized_want_recommend_reason_reason'])]
qs_new['tokenized_want_recommend_reason_reason'] = [' '.join(map(str, l)) for l in qs['tokenized_want_recommend_reason_reason']]
qs_new
print(qs_new['tokenized_want_recommend_reason_reason'])

5                                         男女 比 おかしい 女性 少ない
7                       正直 周り 常連 雰囲気 参加 溶け込む にくい グループ 生まれる
11                            主催 司会 雑 かつ 暴言 聞こえる 程度 ボソッ 言う
12                                                 色々 おかしい
16                                                半分 全員 話す
                               ...                        
20301                                         自己 紹介 カード 微妙
20305                                        相手 少ない 暇 さみしい
20307              予約 確認 メール 届く 用意 あまり 対応 参加 辞退 申し出る 悪い 対応
20308                            段取り あまり 人数 割合 合う 少 人数 過ぎる
20311    女性 完売 謳う いざ 会場 行く 男性 女性 のみ 女性 完売 多数 参加 匂わせる おく...
Name: tokenized_want_recommend_reason_reason, Length: 6617, dtype: object


In [15]:
data_satis = qs_new.tokenized_want_recommend_reason_reason.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data_satis))
data_words[:5]

[['男女', 'おかしい', '女性', '少ない'],
 ['正直', '周り', '常連', '雰囲気', '参加', '溶け込む', 'にくい', 'クルーフ', '生まれる'],
 ['主催', '司会', 'かつ', '暴言', '聞こえる', '程度', 'ホソッ', '言う'],
 ['色々', 'おかしい'],
 ['半分', '全員', '話す']]

In [16]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[data_words[0]]])

['男女', 'おかしい', '女性', '少ない']


In [17]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [18]:
data_words_bigrams = make_bigrams(data_words)
nlp = spacy.load('ja_ginza', disable=['parser', 'ner'])
nlp

In [19]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(0, 1), (21, 1)],
 [(22, 1), (23, 1), (24, 1)],
 [(2, 1), (25, 1), (26, 1), (27, 1)],
 [(28, 1), (29, 1)],
 [(30, 1), (31, 1), (32, 1), (33, 1), (34, 1)],
 [(35, 1), (36, 1)],
 [(1, 1),
  (3, 1),
  (25, 1),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 2),
  (48, 1),
  (49, 1)],
 [(50, 1), (51, 1), (52, 1)],
 [(1, 1), (2, 1), (53, 1)],
 [(54, 2)],
 [(55, 1)],
 [(56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)],
 [(62, 1)],
 [(25, 1),
  (27, 1),
  (47, 1),
  (56, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 2),
  (69, 1),
  (70, 1)],
 [(2, 1), (25, 1)],
 [(47, 1),
  (51, 2),
  (53, 1),
  (66, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1)],
 [(3, 1), (25, 1), (77, 1), (78, 1), (79, 1)

In [20]:
size_id2word= [[(id2word[id], freq) for id, freq in cp] for cp in corpus]

In [21]:
print("Vocabulary size: {}".format(len(id2word)))
print("Corpus size: {}".format(len(corpus)))

Vocabulary size: 4557
Corpus size: 6617


In [22]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                           id2word=id2word, 
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model

In [23]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.042*"書く" + 0.032*"連絡" + 0.032*"全員" + 0.031*"くる" + 0.025*"進行" + 0.025*"交換" '
  '+ 0.022*"司会" + 0.020*"レヘル" + 0.016*"低い" + 0.014*"最初"'),
 (1,
  '0.061*"男性" + 0.056*"女性" + 0.033*"スタッフ" + 0.032*"男女" + 0.031*"悪い" + '
  '0.028*"多い" + 0.023*"話す" + 0.021*"行く" + 0.020*"対応" + 0.018*"わかる"'),
 (2,
  '0.046*"会場" + 0.034*"たけ" + 0.032*"言う" + 0.027*"狭い" + 0.021*"比率" + 0.020*"食事" '
  '+ 0.019*"無い" + 0.018*"テーフル" + 0.016*"トリンク" + 0.015*"出る"'),
 (3,
  '0.141*"人数" + 0.137*"少ない" + 0.118*"参加" + 0.086*"すきる" + 0.034*"料理" + '
  '0.018*"高い" + 0.017*"内容" + 0.013*"違う" + 0.013*"会話" + 0.012*"出来る"'),
 (4,
  '0.035*"イヘント" + 0.029*"開催" + 0.024*"という" + 0.021*"コン" + 0.019*"運営" + '
  '0.017*"合う" + 0.017*"中止" + 0.016*"まて" + 0.015*"返金" + 0.014*"主催"')]


In [24]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.9974638758835095

Coherence Score:  0.40465876341066737


In [25]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.210030 -0.311018       1        1  28.530014
3      0.369063 -0.113460       2        1  24.999727
4     -0.060251  0.225574       3        1  20.892099
2     -0.050732  0.112112       4        1  14.386328
0     -0.048050  0.086792       5        1  11.191836, topic_info=     Category         Freq Term        Total  loglift  logprob
25    Default  1864.000000   人数  1864.000000  30.0000  30.0000
2     Default  1808.000000  少ない  1808.000000  29.0000  29.0000
6     Default  1560.000000   参加  1560.000000  28.0000  28.0000
53    Default  1145.000000  すきる  1145.000000  27.0000  27.0000
47    Default   920.000000   男性   920.000000  26.0000  26.0000
...       ...          ...  ...          ...      ...      ...
277    Topic5    39.890583   全体    40.654209   2.1710  -5.0013
1128   Topic5    38.199902  きつい    38.988216   2.1696  -5.0446
265    Topic5    35.906864  伝える    36.664093   2.1691  -5.1065
280    Topic5    37.413239   恋愛    38.206345   2.1690  -5.0654
1297   Topic5    35.310635  思える    36.069145   2.1687  -5.1232

[181 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
345       1  0.998566    あまり
472       2  0.990842   あまりに
500       2  0.990543     いく
293       5  0.978841    いける
0         4  0.996116   おかしい
...     ...       ...    ...
149       4  0.998201     食事
150       4  0.991701    飲み物
344       4  0.991967     飲む
136       4  0.983377  飲む_放題
111       2  0.994932     高い

[151 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 3, 1])

In [26]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words):
    sent_topics_df = pd.DataFrame()
    
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_contribution', 'Topic_Keywords']
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [27]:
categories_df = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words)
df_dominant_categories = categories_df.reset_index()
df_dominant_categories.columns = ['Document_No', 'Dominant_Categories', 'Category_perc_contrib', 'Keywords', 'Overall_Satisfaction_Reason']
df_dominant_categories

,Document_No,Dominant_Categories,Category_perc_contrib,Keywords,Overall_Satisfaction_Reason
0,0,1.0,0.3499,"男性, 女性, スタッフ, 男女, 悪い, 多い, 話す, 行く, 対応, わかる","[男女, おかしい, 女性, 少ない]"
1,1,3.0,0.5189,"人数, 少ない, 参加, すきる, 料理, 高い, 内容, 違う, 会話, 出来る","[正直, 周り, 常連, 雰囲気, 参加, 溶け込む, にくい, クルーフ, 生まれる]"
2,2,0.0,0.3026,"書く, 連絡, 全員, くる, 進行, 交換, 司会, レヘル, 低い, 最初","[主催, 司会, かつ, 暴言, 聞こえる, 程度, ホソッ, 言う]"
3,3,1.0,0.3255,"男性, 女性, スタッフ, 男女, 悪い, 多い, 話す, 行く, 対応, わかる","[色々, おかしい]"
4,4,1.0,0.3014,"男性, 女性, スタッフ, 男女, 悪い, 多い, 話す, 行く, 対応, わかる","[半分, 全員, 話す]"
...,...,...,...,...,...
6612,6612,1.0,0.3013,"男性, 女性, スタッフ, 男女, 悪い, 多い, 話す, 行く, 対応, わかる","[自己, 紹介, カート, 微妙]"
6613,6613,1.0,0.3758,"男性, 女性, スタッフ, 男女, 悪い, 多い, 話す, 行く, 対応, わかる","[相手, 少ない, さみしい]"
6614,6614,1.0,0.4211,"男性, 女性, スタッフ, 男女, 悪い, 多い, 話す, 行く, 対応, わかる","[予約, 確認, メール, 届く, 用意, あまり, 対応, 参加, 辞退, 申し出る, 悪..."
6615,6615,1.0,0.3470,"男性, 女性, スタッフ, 男女, 悪い, 多い, 話す, 行く, 対応, わかる","[段取り, あまり, 人数, 割合, 合う, 人数, 過きる]"


In [32]:
def qs_categories(model_type=lda_model, corpus=corpus, texts=data_words):
    qs_cat_df = pd.DataFrame()
    for i, row in enumerate(model_type[corpus]):
        row = sorted(row[0], key= lambda x: (x[1]), reverse=True)
        #print(row)
        for j, (topic_num, prop_topic)in enumerate(row):
            #print(topic_num, prop_topic)
            if j == 0:
                wp = model_type.show_topic(topic_num)
                topic_keywords = " ".join([word for word, prop in wp])
                #print(topic_keywords)
                qs_cat_df = qs_cat_df.append(pd.Series([int(topic_num), round(prop_topic,4), round(prop_topic,4), round(prop_topic,4),round(prop_topic,4), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    qs_cat_df.columns = ['Dominant_Topic', 'Perc_Dom_Topic_0', 'Perc_Dom_Topic_1', 'Perc_Dom_Topic_2', 'Perc_Dom_Topic_3', 'Perc_Dom_Topic_4', 'Topic_Keywords']
    contents = pd.Series(texts)
    qs_cat_df = pd.concat([qs_cat_df, contents], axis=1)
    return(qs_cat_df)

In [33]:
categories_df = qs_categories(model_type=lda_model, corpus=corpus, texts=data_words)
questionnaire_cat = categories_df.reset_index()
questionnaire_cat.columns = ['Document_No', 'Dominant_Topic', 'Perc_Topic_0', 'Perc_Topic_1', 'Perc_Topic_2', 'Perc_Topic_3', 'Perc_Topic_4', 'Dom_Top_Keywords', 'Overall_Satisfaction_Reason']
questionnaire_cat

,Document_No,Dominant_Topic,Perc_Topic_0,Perc_Topic_1,Perc_Topic_2,Perc_Topic_3,Perc_Topic_4,Dom_Top_Keywords,Overall_Satisfaction_Reason
0,0,1.0,0.3499,0.3499,0.3499,0.3499,0.3499,男性 女性 スタッフ 男女 悪い 多い 話す 行く 対応 わかる,"[男女, おかしい, 女性, 少ない]"
1,1,3.0,0.5189,0.5189,0.5189,0.5189,0.5189,人数 少ない 参加 すきる 料理 高い 内容 違う 会話 出来る,"[正直, 周り, 常連, 雰囲気, 参加, 溶け込む, にくい, クルーフ, 生まれる]"
2,2,0.0,0.3026,0.3026,0.3026,0.3026,0.3026,書く 連絡 全員 くる 進行 交換 司会 レヘル 低い 最初,"[主催, 司会, かつ, 暴言, 聞こえる, 程度, ホソッ, 言う]"
3,3,1.0,0.3255,0.3255,0.3255,0.3255,0.3255,男性 女性 スタッフ 男女 悪い 多い 話す 行く 対応 わかる,"[色々, おかしい]"
4,4,1.0,0.3014,0.3014,0.3014,0.3014,0.3014,男性 女性 スタッフ 男女 悪い 多い 話す 行く 対応 わかる,"[半分, 全員, 話す]"
...,...,...,...,...,...,...,...,...,...
6612,6612,1.0,0.3013,0.3013,0.3013,0.3013,0.3013,男性 女性 スタッフ 男女 悪い 多い 話す 行く 対応 わかる,"[自己, 紹介, カート, 微妙]"
6613,6613,1.0,0.3758,0.3758,0.3758,0.3758,0.3758,男性 女性 スタッフ 男女 悪い 多い 話す 行く 対応 わかる,"[相手, 少ない, さみしい]"
6614,6614,1.0,0.4211,0.4211,0.4211,0.4211,0.4211,男性 女性 スタッフ 男女 悪い 多い 話す 行く 対応 わかる,"[予約, 確認, メール, 届く, 用意, あまり, 対応, 参加, 辞退, 申し出る, 悪..."
6615,6615,1.0,0.3470,0.3470,0.3470,0.3470,0.3470,男性 女性 スタッフ 男女 悪い 多い 話す 行く 対応 わかる,"[段取り, あまり, 人数, 割合, 合う, 人数, 過きる]"
